# Beer Data File Clean Up

In [1]:
#Import Dependencies
import csv
import pandas as pd
import requests
import json
from config import gkey
from pprint import pprint
import gmaps
import numpy as np

In [2]:
#Import Beer Review Data
beer_data=pd.read_csv('../beer_reviews.csv', encoding = "ISO-8859-1")

#Preview the data
beer_data.head(10)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,1075,Caldera Brewing Company,1325524659,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,1075,Caldera Brewing Company,1318991115,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,1075,Caldera Brewing Company,1306276018,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,1075,Caldera Brewing Company,1290454503,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,1075,Caldera Brewing Company,1285632924,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [3]:
#Group the data by beer name, brewery, and beer type
beer = beer_data.groupby(["beer_name","brewery_name","beer_style"])

# Sort Data by the Beer's Average Review Values and Total Review Counts

In [4]:
#Create a data frame for average ratings based on individual beers
beer_df = pd.DataFrame(beer.mean())
ind_beer_rating_df = beer_df.reset_index(["brewery_name", "beer_style"])

#Change the id column back to an integer for merging later
ind_beer_rating_df.beer_beerid = ind_beer_rating_df.beer_beerid.astype(int)

#Display the data frame
ind_beer_rating_df

,brewery_name,beer_style,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
beer_name,,,,,,,,,,,
! (Old Ale),NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.000000,4.500000,4.000000,4.000000,4.500000,8.20,57645
"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.000000,4.000000,4.000000,4.000000,4.400000,10.00,49236
"""100"" Pale Ale",Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.000000,4.000000,3.500000,4.000000,4.000000,6.60,50696
"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.000000,3.500000,4.250000,4.750000,4.500000,9.00,44353
"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.000000,3.000000,3.000000,3.166667,3.000000,4.80,30080
"""4"" Horse Oatmeal Stout",Thoroughbreds Grill & Brewing,Oatmeal Stout,9819.0,1.126683e+09,4.000000,3.500000,4.000000,3.250000,3.250000,NaN,25570
"""400"" Ale",Williamsburg AleWerks,American Brown Ale,14952.0,1.201561e+09,3.333333,3.833333,3.916667,3.750000,3.444444,NaN,36909
"""50"" Golden Anniversary Lager",Mia & Pia's Pizzeria And Brewhouse,American Pale Lager,10434.0,1.100831e+09,4.000000,3.500000,5.000000,3.500000,3.500000,4.00,20819
"""76"" Anniversary Ale",Bootlegger's Brewery,Old Ale,17271.0,1.260115e+09,4.000000,4.166667,3.666667,4.000000,4.166667,7.60,51029


In [5]:
#Create a datafame to hold the total amount of reviews for each beer
beer_review_count = beer_data.groupby(['beer_name', 'beer_beerid']).count()
beer_review_count = beer_review_count[['review_overall']]
beer_review_count = beer_review_count.reset_index()
beer_review_count.head(10)

,beer_name,beer_beerid,review_overall
0,! (Old Ale),57645,1
1,"""100""",49236,5
2,"""100"" Pale Ale",50696,1
3,"""12"" Belgian Golden Strong Ale",44353,2
4,"""33"" Export",30080,3
5,"""4"" Horse Oatmeal Stout",25570,2
6,"""400"" Ale",36909,18
7,"""50"" Golden Anniversary Lager",20819,1
8,"""76"" Anniversary Ale",51029,3
9,"""76"" Anniversary Ale With English Hops",55510,1


In [6]:
#Merge average review dataframe and review count dataframe
beer_merged_data = pd.merge(ind_beer_rating_df, beer_review_count, on='beer_beerid', how='outer')
beer_merged_data = beer_merged_data.rename(columns={'review_overall_x': 'average_review', 'review_overall_y': 'review_count'})

#Preview Dataframe
beer_merged_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
0,NÃ¤rke Kulturbryggeri AB,Old Ale,10902.0,1.271638e+09,4.0,4.5,4.00,4.000000,4.5,8.2,57645,! (Old Ale),1.0
1,The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,1.254407e+09,4.0,4.0,4.00,4.000000,4.4,10.0,49236,"""100""",5.0
2,Aviator Brewing Company,American IPA,18635.0,1.245965e+09,4.0,4.0,3.50,4.000000,4.0,6.6,50696,"""100"" Pale Ale",1.0
3,Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,1.245585e+09,4.0,3.5,4.25,4.750000,4.5,9.0,44353,"""12"" Belgian Golden Strong Ale",2.0
4,Baltika-Baku,Dortmunder / Export Lager,13640.0,1.200595e+09,3.0,3.0,3.00,3.166667,3.0,4.8,30080,"""33"" Export",3.0


In [7]:
#Sort the merged dataframe by review count
beer_sorted_data = beer_merged_data.sort_values('review_count', ascending=False)
beer_sorted_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
947,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093,90 Minute IPA,3290.0
42967,North Coast Brewing Co.,Russian Imperial Stout,112.0,1.218958e+09,4.174221,4.198650,4.373995,4.231758,4.342173,9.0,412,Old Rasputin Russian Imperial Stout,3111.0
53469,Sierra Nevada Brewing Co.,American IPA,140.0,1.211116e+09,4.168667,4.082333,4.225167,4.076000,4.190000,6.8,1904,Sierra Nevada Celebration Ale,3000.0
60848,"Bell's Brewery, Inc.",American IPA,287.0,1.221429e+09,4.329729,4.266496,4.153959,4.138746,4.318915,7.0,1093,Two Hearted Ale,2728.0
56517,Stone Brewing Co.,American Double / Imperial IPA,147.0,1.215671e+09,4.161612,4.341531,4.179179,4.183432,4.351331,7.7,4083,Stone Ruination IPA,2704.0


In [8]:
#Create bins based on the fibonacci sequence in order to remove beers with a lower review count
bins = [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
review_count_slices = ['1', '2', '3', '4-5', '6-8', '9-13', '14-21', '22-34', '35-55', '56-89', '90-144', '145-233', '234-377', '378-610', '611-987', '988-1597', '1598-2584', '2585 and up']
beer_sorted_data['review_count_slices'] = pd.cut(beer_sorted_data['review_count'], bins, labels=review_count_slices)
beer_sorted_data.head()

,brewery_name,beer_style,brewery_id,review_time,average_review,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count,review_count_slices
947,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,1.212149e+09,4.145745,4.213526,4.192097,4.181763,4.325988,9.0,2093,90 Minute IPA,3290.0,2585 and up
42967,North Coast Brewing Co.,Russian Imperial Stout,112.0,1.218958e+09,4.174221,4.198650,4.373995,4.231758,4.342173,9.0,412,Old Rasputin Russian Imperial Stout,3111.0,2585 and up
53469,Sierra Nevada Brewing Co.,American IPA,140.0,1.211116e+09,4.168667,4.082333,4.225167,4.076000,4.190000,6.8,1904,Sierra Nevada Celebration Ale,3000.0,2585 and up
60848,"Bell's Brewery, Inc.",American IPA,287.0,1.221429e+09,4.329729,4.266496,4.153959,4.138746,4.318915,7.0,1093,Two Hearted Ale,2728.0,2585 and up
56517,Stone Brewing Co.,American Double / Imperial IPA,147.0,1.215671e+09,4.161612,4.341531,4.179179,4.183432,4.351331,7.7,4083,Stone Ruination IPA,2704.0,2585 and up


In [9]:
#Create a dataframe to be able to see how many beers are in each bin
review_1 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1', :]
review_1_count = review_1['review_count'].count()
review_2 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2', :]
review_2_count = review_2['review_count'].count()
review_3 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '3', :]
review_3_count = review_3['review_count'].count()
review_4_5 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '4-5', :]
review_4_5_count = review_4_5['review_count'].count()
review_6_8 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '6-8', :]
review_6_8_count = review_6_8['review_count'].count()
review_9_13 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '9-13', :]
review_9_13_count = review_9_13['review_count'].count()
review_14_21 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '14-21', :]
review_14_21_count = review_14_21['review_count'].count()
review_22_34 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '22-34', :]
review_22_34_count = review_22_34['review_count'].count()
review_35_55 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '35-55', :]
review_35_55_count = review_35_55['review_count'].count()
review_56_89 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '56-89', :]
review_56_89_count = review_56_89['review_count'].count()
review_90_144 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '90-144', :]
review_90_144_count = review_90_144['review_count'].count()
review_145_233 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '145-233', :]
review_145_233_count = review_145_233['review_count'].count()
review_234_377 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '234-377', :]
review_234_377_count = review_234_377['review_count'].count()
review_378_610 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '378-610', :]
review_378_610_count = review_378_610['review_count'].count()
review_611_987 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '611-987', :]
review_611_987_count = review_611_987['review_count'].count()
review_988_1597 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '988-1597', :]
review_988_1597_count = review_988_1597['review_count'].count()
review_1598_2584 = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '1598-2584', :]
review_1598_2594_count = review_1598_2584['review_count'].count()
review_2585_andup = beer_sorted_data.loc[beer_sorted_data["review_count_slices"] == '2585 and up', :]
review_2585_andup_count = review_2585_andup['review_count'].count()



review_count_df = pd.DataFrame({'review_count': [review_1_count, review_2_count, review_3_count, review_4_5_count, review_6_8_count, review_9_13_count, review_14_21_count, review_22_34_count, review_35_55_count, review_56_89_count, review_90_144_count, review_145_233_count, review_234_377_count, review_378_610_count, review_611_987_count, review_988_1597_count, review_1598_2594_count, review_2585_andup_count]})
review_count_df = review_count_df.rename(index={0 : '1', 1 : '2', 2 : '3', 3 : '4-5', 4 : '6-8', 5 : '9-13', 6 : '14-21', 7 : '22-34', 8 : '35-55',
                                                9: '56-89', 10: '90-144', 11: '145-233', 12: '234-377', 13: '378-610', 14: '611-987', 15: '988-1597', 16: '1598-2584', 17: '2585 and up'})
review_count_df


,review_count
1,23805
2,10447
3,5785
4-5,6373
6-8,4609
9-13,3688
14-21,2827
22-34,2191
35-55,1743
56-89,1422


In [10]:
#Remove beers that have less than 9 total reviews
final_count_df = beer_sorted_data.loc[(beer_sorted_data['review_count_slices'] != '1') &
                                      (beer_sorted_data['review_count_slices'] != '2') &
                                      (beer_sorted_data['review_count_slices'] != '3') &
                                      (beer_sorted_data['review_count_slices'] != '4-5') &
                                      (beer_sorted_data['review_count_slices'] != '6-8'), :]
#Save to csv
final_count_df.to_csv('clean_data/beer_review_data_cleaned.csv')

# Sort Data by Beer Style

In [11]:
#Creat a dataframe for beer count by style
beer_type = beer_df.reset_index()
beer_type_df = beer_type.groupby(['beer_style']).count()
beer_type_df = beer_type_df[['beer_name']]
beer_type_df = beer_type_df.sort_values(['beer_name'], ascending=False)
beer_type_df = beer_type_df.reset_index()
beer_type_df = beer_type_df.rename(columns={'beer_name': 'beer_count'})

#Preview dataframe
beer_type_df.head(10)

,beer_style,beer_count
0,American IPA,3588
1,American Pale Ale (APA),3320
2,American Amber / Red Ale,2489
3,American Porter,2181
4,Fruit / Vegetable Beer,1833
5,Hefeweizen,1713
6,American Double / Imperial IPA,1546
7,English Bitter,1486
8,Euro Pale Lager,1479
9,American Stout,1418


In [12]:
#Create a dataframe for review count by stle
beer_review_style = beer_data.groupby(['beer_style']).count()
beer_review_style = beer_review_style[['review_overall']]
beer_review_style = beer_review_style.sort_values(["review_overall"],ascending=False)
beer_review_style = beer_review_style.reset_index()
beer_review_style = beer_review_style.rename(columns={'review_overall': 'review_count'})

#Preview the dataframe
beer_review_style.head(10)

,beer_style,review_count
0,American IPA,117586
1,American Double / Imperial IPA,85977
2,American Pale Ale (APA),63469
3,Russian Imperial Stout,54129
4,American Double / Imperial Stout,50705
5,American Porter,50477
6,American Amber / Red Ale,45751
7,Belgian Strong Dark Ale,37743
8,Fruit / Vegetable Beer,33861
9,American Strong Ale,31945


In [13]:
#Create a dataframe for the average overall rating by style
style_means = beer_data.groupby(['beer_style']).review_overall.mean()
beer_style_means_df = pd.DataFrame(style_means)
beer_style_means_df = beer_style_means_df.sort_values(['review_overall'], ascending=False)
beer_style_means_df = beer_style_means_df.reset_index()
beer_style_means_df = beer_style_means_df.rename(columns={'review_overall': 'review_avg'})

beer_style_means_df.head(10)

,beer_style,review_avg
0,American Wild Ale,4.093262
1,Gueuze,4.086287
2,Quadrupel (Quad),4.071630
3,Lambic - Unblended,4.048923
4,American Double / Imperial Stout,4.029820
5,Russian Imperial Stout,4.023084
6,Weizenbock,4.007969
7,American Double / Imperial IPA,3.998017
8,Flanders Red Ale,3.992722
9,Rye Beer,3.981737


In [14]:
beer_style_df = pd.merge(beer_type_df, beer_review_style, on='beer_style', how='outer')
beer_style_df = pd.merge(beer_style_df, beer_style_means_df, on='beer_style', how='outer')

beer_style_df.head()

,beer_style,beer_count,review_count,review_avg
0,American IPA,3588,117586,3.965221
1,American Pale Ale (APA),3320,63469,3.852306
2,American Amber / Red Ale,2489,45751,3.781010
3,American Porter,2181,50477,3.895735
4,Fruit / Vegetable Beer,1833,33861,3.415124


In [15]:
#Save to csv
beer_style_df.to_csv('clean_data/beer_style_data_cleaned.csv')

# Sort Data by Breweries in the US

In [16]:
#Create a dataframe by brewery and the average review for the beers they make
brew_rating = beer_data.groupby(["brewery_name", "brewery_id"])
average_rating = round(brew_rating['review_overall'].mean(), 2)
brewery_avg_rating_df = pd.DataFrame(average_rating)
brewery_avg_rating_df = brewery_avg_rating_df.reset_index()
brewery_avg_rating_df = brewery_avg_rating_df.rename(columns={'brewery_name': 'Brewery'})

#Preview the dataframe
brewery_avg_rating_df.head()

,Brewery,brewery_id,review_overall
0,'t Hofbrouwerijke,13160,3.70
1,(512) Brewing Company,17863,4.07
2,10 Barrel Brewing Co.,16873,3.73
3,1516 Brewing Company,4473,4.13
4,16 Mile Brewing Company,20688,3.57


In [17]:
#Clean dataframe to remove special characters for the google api call
clean_brew_df = brewery_avg_rating_df[~brewery_avg_rating_df['Brewery'].astype(str).str.contains('[^a-zA-Z0-9 ]', regex=True)]
clean_brew_df = clean_brew_df.reset_index()
clean_brew_df = clean_brew_df.drop(columns='index')

#Preview the dataframe
clean_brew_df.head()

,Brewery,brewery_id,review_overall
0,1516 Brewing Company,4473,4.13
1,16 Mile Brewing Company,20688,3.57
2,1648 Brewing Company Ltd,8396,3.50
3,192 Brewing Company,22972,1.83
4,1st City Brewery and Grill,4437,3.67


In [18]:
#Add columns to fill during the api call
clean_brew_df['latitude'] = ''
clean_brew_df['longitude'] = ''
clean_brew_df['google_rating'] = ''

#Preview the dataframe
clean_brew_df.head()

,Brewery,brewery_id,review_overall,latitude,longitude,google_rating
0,1516 Brewing Company,4473,4.13,,,
1,16 Mile Brewing Company,20688,3.57,,,
2,1648 Brewing Company Ltd,8396,3.50,,,
3,192 Brewing Company,22972,1.83,,,
4,1st City Brewery and Grill,4437,3.67,,,


In [19]:
#API Call
put = "textquery"                                
field = "id,name,rating,geometry/location"
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
count = 0

for index, row in clean_brew_df.iterrows():
    params = {
        "input": row['Brewery'],
        "key": gkey,
        "inputtype": put,
        "fields": field
    }
    count += 1
    top_response = requests.get(base_url, params=params)
    result = top_response.json()

    
    try:
        print(f"Making Google request #{count} for {row['Brewery']}.")
        clean_brew_df.loc[index, 'latitude'] = result['candidates'][0]['geometry']['location']['lat']
        clean_brew_df.loc[index, 'longitude'] = result['candidates'][0]['geometry']['location']['lng']
        clean_brew_df.loc[index, 'google_rating'] = result['candidates'][0]['rating']
    
    except (KeyError, IndexError):
        print(f"No Google result for {row['Brewery']}......Skipping.....")

Making Google request #1 for 1516 Brewing Company.
Making Google request #2 for 16 Mile Brewing Company.
Making Google request #3 for 1648 Brewing Company Ltd.
Making Google request #4 for 192 Brewing Company.
Making Google request #5 for 1st City Brewery and Grill.
No Google result for 1st City Brewery and Grill......Skipping.....
Making Google request #6 for 2 Brothers Brewery.
Making Google request #7 for 21st Amendment Brewery.
Making Google request #8 for 23rd Street Brewery.
Making Google request #9 for 2nd Shift Brewery.
Making Google request #10 for 3 Cordilleras.
No Google result for 3 Cordilleras......Skipping.....
Making Google request #11 for 3 Ravens Brewing.
Making Google request #12 for 3 Stars Brewing Company.
Making Google request #13 for 32 Via Dei Birrai.
Making Google request #14 for 4 Pines Brewing Company.
No Google result for 4 Pines Brewing Company......Skipping.....
Making Google request #15 for 4th Street Brewing Company.
Making Google request #16 for 5 Points

Making Google request #113 for Armadillo Ale Works.
Making Google request #114 for Arsentich.
No Google result for Arsentich......Skipping.....
Making Google request #115 for Artebirraia.
No Google result for Artebirraia......Skipping.....
Making Google request #116 for Artos Breweries Ltd.
No Google result for Artos Breweries Ltd......Skipping.....
Making Google request #117 for Arundel Brewery Limited.
Making Google request #118 for Asahi Breweries Ltd.
No Google result for Asahi Breweries Ltd......Skipping.....
Making Google request #119 for Ascot Ales.
Making Google request #120 for Asher Brewing Company.
Making Google request #121 for Asheville Brewing Company.
Making Google request #122 for Asmara Breweries.
No Google result for Asmara Breweries......Skipping.....
Making Google request #123 for Ass Clown Brewing.
Making Google request #124 for Ass Kisser Ales.
No Google result for Ass Kisser Ales......Skipping.....
Making Google request #125 for Athenian Brewery.
Making Google re

Making Google request #219 for Bayou Teche Brewery.
Making Google request #220 for Bayreuther Bierbrauerei.
Making Google request #221 for Beach Brewing Company.
No Google result for Beach Brewing Company......Skipping.....
Making Google request #222 for Beach Chalet Brewery and Restaurant.
Making Google request #223 for Bear Brewing.
Making Google request #224 for Bear Creek Brewing Company.
Making Google request #225 for Beartown Brewery.
Making Google request #226 for Beauce Broue.
No Google result for Beauce Broue......Skipping.....
Making Google request #227 for Beaver Beer Company.
Making Google request #228 for Beaver Brewing Company.
Making Google request #229 for Beaver Creek Brewery.
Making Google request #230 for Beba.
No Google result for Beba......Skipping.....
Making Google request #231 for Becker Brewing Company.
Making Google request #232 for Bedele Brewery.
Making Google request #233 for Bee Creek Brewery.
No Google result for Bee Creek Brewery......Skipping.....
Makin

Making Google request #333 for Birrificio Di Como.
Making Google request #334 for Birrificio Doppio Malto.
No Google result for Birrificio Doppio Malto......Skipping.....
Making Google request #335 for Birrificio Emiliano.
Making Google request #336 for Birrificio Freccia.
No Google result for Birrificio Freccia......Skipping.....
Making Google request #337 for Birrificio Geco.
Making Google request #338 for Birrificio Hibu.
Making Google request #339 for Birrificio Indipendente Elav.
Making Google request #340 for Birrificio Italiano.
Making Google request #341 for Birrificio Lambrate.
Making Google request #342 for Birrificio Lariano.
Making Google request #343 for Birrificio Le Baladin.
No Google result for Birrificio Le Baladin......Skipping.....
Making Google request #344 for Birrificio Lodigiano.
Making Google request #345 for Birrificio Menaresta.
Making Google request #346 for Birrificio Montegioco.
Making Google request #347 for Birrificio Pausa Cafe.
No Google result for Birr

Making Google request #448 for Brasserie Artisanale du Centre.
No Google result for Brasserie Artisanale du Centre......Skipping.....
Making Google request #449 for Brasserie Augrenoise.
No Google result for Brasserie Augrenoise......Skipping.....
Making Google request #450 for Brasserie Authentique.
No Google result for Brasserie Authentique......Skipping.....
Making Google request #451 for Brasserie Bailleux.
Making Google request #452 for Brasserie Battin.
No Google result for Brasserie Battin......Skipping.....
Making Google request #453 for Brasserie Belgh Brasse.
No Google result for Brasserie Belgh Brasse......Skipping.....
Making Google request #454 for Brasserie Bofferding.
Making Google request #455 for Brasserie Bourganel.
Making Google request #456 for Brasserie Britt de Bretagne.
Making Google request #457 for Brasserie Brootcoorens.
Making Google request #458 for Brasserie Cantillon.
Making Google request #459 for Brasserie Caracole.
Making Google request #460 for Brasser

...

Making Google request #2921 for Sternquell Brauerei Plauen.
Making Google request #2922 for Stevens Point Brewery.
Making Google request #2923 for Stevns Bryghus.
Making Google request #2924 for Stewart Brewing Ltd.
Making Google request #2925 for Stieglbrauerei zu Salzburg GmbH.
Making Google request #2926 for Stillwater Artisanal Ales.
No Google result for Stillwater Artisanal Ales......Skipping.....
Making Google request #2927 for Stingray Brewery.
No Google result for Stingray Brewery......Skipping.....
Making Google request #2928 for Stockade Brewery.
Making Google request #2929 for Stone Arch Brewpub.
Making Google request #2930 for Stone City Brewing.
Making Google request #2931 for Stonecutters Brewhouse.
No Google result for Stonecutters Brewhouse......Skipping.....
Making Google request #2932 for Stonehouse Brewery.
Making Google request #2933 for Stoney Creek Brewing Company.
Making Google request #2934 for Storm Brewing.
Making Google request #2935 for Straight To Ale.
Maki

Making Google request #3031 for The Cellar Grill.
No Google result for The Cellar Grill......Skipping.....
Making Google request #3032 for The Celt Experience Brewery.
Making Google request #3033 for The Civil Life Brewing Company.
Making Google request #3034 for The Commons Brewery.
Making Google request #3035 for The Defiant Brewing Company.
Making Google request #3036 for The Dodging Duck Brewhaus.
Making Google request #3037 for The Doodle Brewing Company.
No Google result for The Doodle Brewing Company......Skipping.....
Making Google request #3038 for The Famous Railway Tavern Brewery.
No Google result for The Famous Railway Tavern Brewery......Skipping.....
Making Google request #3039 for The Fat Cat Beer Company.
No Google result for The Fat Cat Beer Company......Skipping.....
Making Google request #3040 for The Gahan House.
Making Google request #3041 for The Greenhouse.
Making Google request #3042 for The Grey Parrot.
No Google result for The Grey Parrot......Skipping.....
Ma

Making Google request #3142 for Traverse Brewing Company.
No Google result for Traverse Brewing Company......Skipping.....
Making Google request #3143 for Trebjesa Brewery.
Making Google request #3144 for Tree Brewing.
Making Google request #3145 for Triangle Brewing Company.
No Google result for Triangle Brewing Company......Skipping.....
Making Google request #3146 for Trinidad Brewing Company.
Making Google request #3147 for Trinity Brewhouse.
Making Google request #3148 for Trinity Brewing Company.
Making Google request #3149 for Triple 7 Restaurant and Brewery.
Making Google request #3150 for Triple J Chophouse And Brew Co.
Making Google request #3151 for Triple Voodoo Brewing.
Making Google request #3152 for Triple fff Brewing Company.
Making Google request #3153 for Triscele Srl.
No Google result for Triscele Srl......Skipping.....
Making Google request #3154 for Triton Brewing Company.
Making Google request #3155 for Triumph Brewing Company.
No Google result for Triumph Brewing

Making Google request #3245 for Voronezhsky Brewery.
No Google result for Voronezhsky Brewery......Skipping.....
Making Google request #3246 for Vulkan Brauhaus GmbH.
No Google result for Vulkan Brauhaus GmbH......Skipping.....
Making Google request #3247 for Wabash Valley Malt Beverage Company.
No Google result for Wabash Valley Malt Beverage Company......Skipping.....
Making Google request #3248 for Wachusett Brewing Company.
Making Google request #3249 for Wagner Brau.
No Google result for Wagner Brau......Skipping.....
Making Google request #3250 for Wagner Valley Brewery.
Making Google request #3251 for Wagtail Brewery.
No Google result for Wagtail Brewery......Skipping.....
Making Google request #3252 for Waiheke Island Brewery Ltd.
No Google result for Waiheke Island Brewery Ltd......Skipping.....
Making Google request #3253 for Waimea Brewing Company.
Making Google request #3254 for Wakonda Brewing Company.
Making Google request #3255 for Walkabout Brewery.
Making Google reques

Making Google request #3351 for Wolf Beer Company.
No Google result for Wolf Beer Company......Skipping.....
Making Google request #3352 for Wolf Brewing Company.
Making Google request #3353 for Wolf Creek Brewing.
No Google result for Wolf Creek Brewing......Skipping.....
Making Google request #3354 for Wolf Hills Brewing Company.
Making Google request #3355 for Wolf Rock Brewing Company.
No Google result for Wolf Rock Brewing Company......Skipping.....
Making Google request #3356 for Wolverine State Brewing Company.
Making Google request #3357 for Wood Brewery Ltd.
No Google result for Wood Brewery Ltd......Skipping.....
Making Google request #3358 for Wooden Hand Brewery.
Making Google request #3359 for Wooden Shoe Brewing Company.
No Google result for Wooden Shoe Brewing Company......Skipping.....
Making Google request #3360 for Woodstock Inn Brewery.
Making Google request #3361 for Woodward Avenue Brewers.
Making Google request #3362 for Woof Breweries.
No Google result for Woof B

In [20]:
clean_brew_df['latitude'] = clean_brew_df['latitude'].replace('', np.nan)
clean_brew_df = clean_brew_df.dropna()
clean_brew_df


,Brewery,brewery_id,review_overall,latitude,longitude,google_rating
0,1516 Brewing Company,4473,4.13,48.203384,16.373,4.4
1,16 Mile Brewing Company,20688,3.57,38.678818,-75.3781,4.6
2,1648 Brewing Company Ltd,8396,3.50,50.926026,0.166076,0
3,192 Brewing Company,22972,1.83,47.756803,-122.242,4.1
5,2 Brothers Brewery,16847,3.69,-37.942725,145.069,4.6
6,21st Amendment Brewery,735,3.70,37.782470,-122.393,4.3
7,23rd Street Brewery,9167,3.90,38.943023,-95.2811,4.3
8,2nd Shift Brewery,24428,3.77,38.621493,-90.2808,4.8
10,3 Ravens Brewing,4778,3.55,-37.755695,145.027,4.7
11,3 Stars Brewing Company,25696,3.17,38.967491,-77.0126,4.6


In [21]:
#Remove all breweries that are not in the US by lat and lon
us_brewery_df = clean_brew_df.loc[(clean_brew_df['latitude'] <= 49.3457868) & 
                            (clean_brew_df['latitude'] >= 24.7433195) &
                            (clean_brew_df['longitude'] <= -66.9513812) &
                            (clean_brew_df['longitude'] >= -124.7844079), :]

#Preview the dataframe
us_brewery_df

,Brewery,brewery_id,review_overall,latitude,longitude,google_rating
1,16 Mile Brewing Company,20688,3.57,38.678818,-75.3781,4.6
3,192 Brewing Company,22972,1.83,47.756803,-122.242,4.1
6,21st Amendment Brewery,735,3.70,37.782470,-122.393,4.3
7,23rd Street Brewery,9167,3.90,38.943023,-95.2811,4.3
8,2nd Shift Brewery,24428,3.77,38.621493,-90.2808,4.8
11,3 Stars Brewing Company,25696,3.17,38.967491,-77.0126,4.6
14,4th Street Brewing Company,3087,3.58,45.479688,-122.402,5
16,5 Rabbit Cerveceria,25544,3.60,41.757960,-87.7802,4.7
18,508 Gastrobrewery,25860,4.25,40.725724,-74.0094,3.4
21,7 Seas Brewery and Taproom,20658,3.58,47.242177,-122.439,4.6


In [22]:
#Save the US api call data as csv
us_brewery_df.to_csv('clean_data/us_brewery_clean_data.csv')